# NBA 2024-25: Utilizing Roles
## Notebook 05: Merge 2024-25 Game Logs & 2024-25 Season Averages
This notebook merges player-level game logs and season-level averages to ensure that aggregations are correct in later player-level analysis.

In [2]:
# Import libraries
import pandas as pd
from unidecode import unidecode
import re

___
## Load

In [4]:
# Load game logs and season averages
game_logs_2024_25 = pd.read_parquet(r"C:\Users\dylan\OneDrive\Documents\Portfolio_Projects\NBA_2024_25_utilizing_roles\03_python_outputs\Merged_Player_Team_GameLogs_2024_25.parquet")
season_averages_2024_25 = pd.read_parquet(r"C:\Users\dylan\OneDrive\Documents\Portfolio_Projects\NBA_2024_25_utilizing_roles\03_python_outputs\NBA_Per_Game_2024_25.parquet")

In [5]:
# Inspect game logs columns
game_logs_2024_25.columns

Index(['Player_Name', 'Player_ID', 'GAME_ID', 'GAME_DATE', 'MATCHUP', 'WL',
       'MIN', 'FGM', 'FGA', 'FTA', 'TOV', 'REB', 'AST', 'PTS',
       'TEAM_ABBREVIATION', 'TEAM_ID', 'TEAM_MIN', 'TEAM_FGA', 'TEAM_FTA',
       'TEAM_TOV', 'USG%', 'PRA'],
      dtype='object')

In [6]:
# Inspect season averages columns
season_averages_2024_25.columns

Index(['Player', 'Season', 'Age', 'Team', 'G', 'Pos', 'PTS', 'TRB', 'AST',
       'USG%'],
      dtype='object')

___
## Clean

In [8]:
# Rename season columns for clarity
season_averages_2024_25 = season_averages_2024_25.rename(columns={
    "Player": "Player_Name",
    "PTS": "season_PTS",
    "TRB": "season_REB",
    "AST": "season_AST",
    "USG%": "season_USG%"
})

In [9]:
# Compute season PRA
season_averages_2024_25["season_PRA"] = (
    season_averages_2024_25["season_PTS"] + season_averages_2024_25["season_REB"] + season_averages_2024_25["season_AST"]
)

In [10]:
# Confirm season columns
season_averages_2024_25.columns

Index(['Player_Name', 'Season', 'Age', 'Team', 'G', 'Pos', 'season_PTS',
       'season_REB', 'season_AST', 'season_USG%', 'season_PRA'],
      dtype='object')

___
## Name Matching

In [12]:
# Confirm number of unique names in game logs
game_logs_2024_25["Player_Name"].nunique()

569

In [13]:
# Confirm number of unique names in season averages
season_averages_2024_25["Player_Name"].nunique()

569

In [14]:
# Export `Player_Name` column from each DataFrame
game_logs_2024_25 = game_logs_2024_25.sort_values("Player_Name")
season_averages_2024_25 = season_averages_2024_25.sort_values("Player_Name")

game_logs_2024_25["Player_Name"].to_csv("game_logs_names.csv", index=False)
season_averages_2024_25["Player_Name"].to_csv("season_averages_names.csv", index=False)

## 🔍

It turns out that the player names from API are slightly different than those from Basketball-Reference. Some names may be missing accents or a suffix, etc.

All player names from both sources were exported, and then I had to manually scan through all 569 names across 2 text files to find out which names had differences.

> For example, in Basketball Reference, his name is written as Dante Exum, but in API, his name is written as Danté Exum. The names must be exactly the same for a successful merge, so I decided to use the API names since this is where I intend to acquire most data from in the future.

This step is as important as it is tedious.

In [16]:
# Load name mapping CSV
cleaned_names = pd.read_csv(r"C:\Users\dylan\OneDrive\Documents\Portfolio_Projects\project11_NBA_all_star_potential\03_python_outputs\name_matching\incorrect_name_mapping.txt")

In [56]:
# All name updates
cleaned_names

,original_name,cleaned_name
0,A.J. Green,AJ Green
1,Alperen Şengün,Alperen Sengun
2,Armel Traoré,Armel Traore
3,Brandon Boston Jr.,Brandon Boston
4,Cui Yongxi,Cui Cui
5,Dante Exum,Danté Exum
6,GG Jackson II,GG Jackson
7,Jeenathan Williams,Nate Williams
8,Jimmy Butler,Jimmy Butler III
9,Kevin Knox,Kevin Knox II


In [18]:
# Strip any whitespace
cleaned_names["original_name"] = cleaned_names["original_name"].str.strip()
cleaned_names["cleaned_name"] = cleaned_names["cleaned_name"].str.strip()

In [19]:
# Create dictionary
name_dict = dict(zip(cleaned_names["original_name"], cleaned_names["cleaned_name"]))

# Apply dictionary to game logs DataFrame
game_logs_2024_25["Player_Name"] = game_logs_2024_25["Player_Name"].replace(name_dict)

# Apply dictionary to season averages DataFrame
season_averages_2024_25["Player_Name"] = season_averages_2024_25["Player_Name"].replace(name_dict)

In [20]:
# Confirm there are 569 names in the intersection of both DataFrames
len(
    set(game_logs_2024_25["Player_Name"]).intersection(
    set(season_averages_2024_25["Player_Name"])
)
)

569

✅ All 569 player names are correct now

___
## Merge

In [22]:
# Merge season averages into game logs
merged_2024_25 = game_logs_2024_25.merge(
    season_averages_2024_25,
    on="Player_Name",
    how="left"
)

In [23]:
# Confirm merged DataFrame columns
merged_2024_25.columns

Index(['Player_Name', 'Player_ID', 'GAME_ID', 'GAME_DATE', 'MATCHUP', 'WL',
       'MIN', 'FGM', 'FGA', 'FTA', 'TOV', 'REB', 'AST', 'PTS',
       'TEAM_ABBREVIATION', 'TEAM_ID', 'TEAM_MIN', 'TEAM_FGA', 'TEAM_FTA',
       'TEAM_TOV', 'USG%', 'PRA', 'Season', 'Age', 'Team', 'G', 'Pos',
       'season_PTS', 'season_REB', 'season_AST', 'season_USG%', 'season_PRA'],
      dtype='object')

___
## Save

In [42]:
# Save to parquet
merged_2024_25.to_parquet("Merged_Player_Team_GameLogs_2024_25_final.parquet")